## Import model

In [79]:
## import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

CUDA = torch.cuda.is_available()
# device = "cpu"
device = torch.device("cuda" if CUDA else "cpu")

## Prep dataset

In [80]:
import sys
# run the below line once only
if "..\\chexnet" not in sys.path:
    sys.path.insert(0,r'..\chexnet')
print(sys.path)

['..\\chexnet', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [81]:
from DatasetGenerator import DatasetGenerator

In [82]:
pathDirData = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\raw_data\archive'
pathFileTrain = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt'
pathFileVal = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt'

transResize = 256
transCrop = 224
trBatchSize = 128
num_class = 14

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

torch.manual_seed(42)

transformList = []
transformList.append(transforms.Resize(transResize))
transformList.append(transforms.RandomResizedCrop(transCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

datasetTrain = DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileTrain, transform=transformSequence)
datasetVal =   DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileVal, transform=transformSequence)
train_loader = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=12, pin_memory=True)
val_loader = DataLoader(dataset=datasetVal, batch_size=trBatchSize, shuffle=False, num_workers=12, pin_memory=True)

Collected 2048 images from C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt
Collected 2048 images from C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt


## Call MaxVit Model

In [83]:
from torchvision.models import vision_transformer

In [84]:
## Initialise first layer
model = vision_transformer.vit_b_16(
    weights = vision_transformer.ViT_B_16_Weights,
    # input_size = (transCrop, transCrop)
)
#  Add the last layer 
model.heads.append(nn.Sequential(
    nn.Linear(in_features=model.heads.head.out_features,out_features= num_class),
    nn.Softmax(dim=1)
))

##  Freeze earlier weights
for name, param in model.named_parameters():
    if 'heads' not in name:
        param.requires_grad= False


## set to cuda
model.cuda()

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

Model Training

In [85]:
## function to calculate the F1 score
def f1_score(tp, fp, fn):
    return 2 * (tp) / (2 * tp + fp + fn)

In [86]:
# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 5, mode = 'min')

# Create a TensorBoard writer
writer = SummaryWriter()

# Train the model
n_epochs = 5
for epoch in range(n_epochs):

    for i, (images, labels, _) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # calculate statistics
        # pred_labels = (nn.Softmax(dim=1)(outputs) > 1/14).long()
        
        tp_array = [0 for x in range(num_class)]
        fp_array = [0 for x in range(num_class)]
        fn_array = [0 for x in range(num_class)]
        pred_labels = (outputs > 1/4).long()
        tp_array += sum(torch.logical_and(pred_labels, labels))
        fp_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), pred_labels))
        fn_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), labels))
        
        writer.add_scalar('Loss/train', loss, epoch * len(train_loader) + i)
        writer.add_scalar('TP_Sum/train', sum(tp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FP_Sum/train', sum(fp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FN_Sum/train', sum(fn_array), epoch * len(train_loader) + i)
        writer.add_scalar('F1_Score/train', f1_score(sum(tp_array), sum(fp_array), sum(fn_array)), epoch * len(train_loader) + i)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)

        # Display
        # if (i + 1) % 100 == 0:
        print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, tp_sum: {:.4f}, fp_sum: {:.4f}, fn_sum: {:.4f}, cumulative_f1_score: {:.4f}".format(epoch + 1, \
                                                                     n_epochs, \
                                                                     i + 1, \
                                                                     len(train_loader), \
                                                                     loss,\
                                                                     sum(tp_array), \
                                                                     sum(fp_array),\
                                                                     sum(fn_array),\
                                                                     f1_score(sum(tp_array), sum(fp_array), sum(fn_array))))
        print("pred_labels\n", pred_labels)
        print("actual labels\n", labels)
    break

Epoch [1/5], Step [1/16], Loss: 0.1885, tp_sum: 0.0000, fp_sum: 3.0000, fn_sum: 75.0000, cumulative_f1_score: 0.0000
pred_labels
 tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
actual labels
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
Epoch [1/5], Step [2/16], Loss: 0.2272, tp_sum: 22.0000, fp_sum: 170.0000, fn_sum: 68.0000, cumulative_f1_score: 0.1560
pred_labels
 tensor([[0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]]